In [24]:
import torch
from utils import *

In [47]:
d = 10

X = np.random.multivariate_normal(np.zeros(d),np.eye(d),100)

mu2 = np.zeros(d)
mu2[0] = 1
mu2[1] = 20


Y = np.random.multivariate_normal(np.zeros(d),np.eye(d),100)



In [48]:
THETA = sample_theta(X,1000)
%timeit project_and_calc_dist(X,Y,THETA,p=2).mean()

13.5 ms ± 164 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Tensor_THETA = torch.Tensor(THETA)


def project_and_calc_dist_torch(X, Y, theta, p):
    # Project data onto theta
    x_proj = torch.matmul(X, theta.T)
    y_proj = torch.matmul(Y, theta.T)

    # Generate quantile positions
    qs = torch.linspace(0, 1, 100, device=X.device)

    # Compute quantiles for x_proj and y_proj
    xp_quantiles = torch.quantile(x_proj, qs, dim=0, interpolation="lower")
    yp_quantiles = torch.quantile(y_proj, qs, dim=0, interpolation="lower")

    # Calculate the p-distance between quantiles
    dist_p = torch.abs(xp_quantiles - yp_quantiles).pow(p)

    return dist_p




In [50]:
%timeit project_and_calc_dist_torch(torch.Tensor(X).to(device),torch.Tensor(Y).to(device),Tensor_THETA.to(device),p=2)

473 μs ± 55.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
